In [ ]:
# Direitos autorais 2024 Google LLC
#
# Licenciado sob a Licença Apache, Versão 2.0 (a "Licença");
# você não pode usar este arquivo exceto em conformidade com a Licença.
# Você pode obter uma cópia da Licença em
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# A menos que exigido pela lei aplicável ou acordado por escrito, o software
# distribuído sob a Licença é distribuído "COMO ESTÁ",
# SEM GARANTIAS OU CONDIÇÕES DE QUALQUER TIPO, expressas ou implícitas.
# Consulte a Licença para o idioma específico que rege as permissões e
# limitações sob a Licença.

# Engenharia de Prompt - Melhores práticas

> ***NOTA:** Este notebook utiliza o modelo generativo PaLM, que atingirá seu [data de descontinuação em outubro de 2024](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text#model_versions). Consulte [este caderno atualizado](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb) para uma versão que usa o modelo Gemini mais recente.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/language/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) |

## Visão geral

Este caderno aborda os fundamentos da engenharia de prompts, incluindo algumas práticas recomendadas.


Saiba mais sobre o design de prompts na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

### Objetivo

Neste caderno, você aprenderá as práticas recomendadas em torno da engenharia de prompts – como projetar prompts para melhorar a qualidade de suas respostas.

Este notebook aborda as seguintes práticas recomendadas para design de prompts

- Seja conciso
- Seja específico e bem definido
- Peça uma tarefa de cada vez
- Transforme tarefas generativas em tarefas de classificação
- Melhore a qualidade da resposta incluindo exemplos

### Custos
Este tutorial usa componentes faturáveis ​​do Google Cloud:

* Estúdio de IA generativa da Vertex AI

Saiba mais sobre [Vertex AI preços](https://cloud.google.com/vertex-ai/pricing),
 use o [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

### Instalar o SDK da Vertex AI


In [ ]:
%pip install --upgrade google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


**Observação:** Ignore os avisos de descontinuação e erros de incompatibilidade relacionados às dependências do pip.

**Colab only:** Execute a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel. Para **Vertex AI Workbench** você pode reiniciar o terminal usando o botão na parte superior.

In [ ]:
# Reinicie automaticamente o kernel após a instalação para que seu ambiente possa acessar os novos pacotes
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Autenticando seu ambiente de notebook

- Se você estiver usando o **Colab** para executar este notebook, execute a célula abaixo e continue.
- Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

- Se você estiver executando este notebook em um ambiente de desenvolvimento local:
  - Instale o [Google Cloud SDK](https://cloud.google.com/sdk).
  - Obtenha credenciais de autenticação. Crie credenciais locais executando o seguinte comando e seguindo o fluxo oauth2 (leia mais sobre o comando [aqui](https://cloud.google.com/sdk/gcloud/reference/beta/auth/application-default/login)):

    ```bash
    gcloud auth application-default login
    ```

### Importar bibliotecas

**Colab only:** Execute a célula a seguir para inicializar o SDK da Vertex AI. Para o Vertex AI Workbench, você não precisa executar isso.


In [ ]:
import vertexai

PROJECT_ID = "qwiklabs-gcp-02-4b320c1cc30e"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
from vertexai.language_models import TextGenerationModel
from vertexai.language_models import ChatModel

### Carregar o modelo

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison")

I0000 00:00:1724617248.435049    5899 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


## Práticas recomendadas de Engenharia de Prompts

A engenharia de prompts trata de como projetar seus prompts para que a resposta seja o que você realmente esperava ver.

A ideia de usar prompts "pouco sofisticados" é minimizar o ruído (informações desnecessárias ou confusas) em seu prompt para reduzir a possibilidade de o LLM (Modelo de Linguagem de Grande Escala)interpretar mal a intenção do prompt. Abaixo estão algumas diretrizes sobre como projetar prompts "pouco sofisticados".

Nesta seção, você abordará as seguintes práticas recomendadas à engenharia de prompts:

* Seja conciso
* Seja específico e bem definido
* Peça uma tarefa de cada vez
* Melhore a qualidade da resposta incluindo exemplos
* Transforme tarefas generativas em tarefas de classificação para melhorar a segurança

### Seja conciso

🛑 Não recomendado. O prompt abaixo é desnecessariamente detalhado.

In [ ]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers? Thank you!"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

 Here are some potential names for a flower shop that specializes in selling bouquets of dried flowers:

- Everlasting Blooms
- Dried Flower Delights
- Nature's Treasures
- Rustic Florals
- Timeless Botanicals
- Floral Keepsakes
- Dried Flower Gallery
- Botanical Memories
- Forever Flowers
- Pressed Petals


✅ Recomendado. O prompt abaixo é direto e conciso.

In [ ]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

 **The Everlasting Bloom**
**Dried Delights**
**Nature's Treasures**
**Rustic Blooms**
**Vintage Florals**
**Dried Floral Creations**
**Preserved Petals**
**Eternal Elegance**
**Botanical Beauties**
**Nature's Keepsakes**


### Seja específico e bem definido

*Suponha* que você queira pensar em maneiras criativas de descrever a Terra.

🛑 Não recomendado. O prompt abaixo é muito genérico.

In [ ]:
prompt = "Tell me about Earth"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

 Earth is the third planet from the Sun and the only astronomical object known to harbor life. While larger than Mercury and Venus, Earth is smaller than Mars and the gas and ice giants of the outer Solar System. Earth is the only planet in our solar system not named after a Greek or Roman deity. Instead, its name comes from the Old English word "erda," which means "ground" or "soil."

Earth is the only planet in the Solar System known to support life. It is the only planet with liquid water on its surface, and it has a breathable atmosphere. Earth is also the only planet with a magnetic field, which protects it from harmful solar radiation.

Earth is a dynamic planet. Its climate is constantly changing, and its surface is constantly being reshaped by geological processes. Earth is also home to a wide variety of life forms, from microscopic bacteria to giant whales.

Here are some of the key facts about Earth:

* **Mass:** 5.972 × 10^24 kilograms
* **Volume:** 1.08321 × 10^12 cubic kil

✅ Recomendado. O prompt abaixo é específico e bem definido.

In [ ]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

 **Earth's unique characteristics that set it apart from other planets include:**

1. **Liquid Water:** Earth is the only known planet in our solar system with liquid water on its surface. This is essential for life as we know it, as water is a key component of all living organisms.

2. **Oxygen-Rich Atmosphere:** Earth's atmosphere is composed of approximately 21% oxygen, which is essential for respiration and the survival of most life forms.

3. **Plate Tectonics:** Earth is the only planet known to have active plate tectonics, which is the movement of the Earth's crust. This process creates mountains, volcanoes, and other geological features, and also plays a role in recycling the Earth's materials.

4. **Magnetic Field:** Earth's magnetic field is generated by the movement of molten iron in the Earth's core. This field protects the planet from harmful solar radiation and cosmic rays, making it possible for life to exist on the surface.

5. **Biodiversity:** Earth is home to an incr

### Peça uma tarefa de cada vez


🛑 Não recomendado. O prompt abaixo tem duas partes para a pergunta que podem ser feitas separadamente.

In [ ]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

 **The best method of boiling water:**

The best method of boiling water depends on the situation and the equipment available. Here are a few common methods:

1. **Electric kettle:** An electric kettle is a convenient and efficient way to boil water. It is quick and easy to use, and it automatically turns off when the water reaches the desired temperature.

2. **Stovetop kettle:** A stovetop kettle is a traditional method of boiling water. It is placed on a stove or cooktop and heated until the water reaches the desired temperature. Stovetop kettles can be made of various materials, such as stainless steel, aluminum, or copper.

3. **Microwave:** A microwave can be used to boil water quickly and easily. Place a microwave-safe container filled with water in the microwave and heat it on high power for 2-3 minutes, or until the water reaches the desired temperature.

4. **Campfire:** If you are outdoors and do not have access to electricity or a stove, you can boil water over a campfire. 

✅ Recomendado. Os prompts abaixo solicitam uma tarefa por vez.

In [ ]:
prompt = "What's the best method of boiling water?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

 The best method of boiling water depends on the specific situation and available resources. Here are a few common methods:

1. **Electric Kettle:**
   - Electric kettles are designed specifically for boiling water and are very efficient.
   - They heat water quickly and automatically turn off when the water reaches the boiling point.
   - Electric kettles are convenient and safe to use.

2. **Stovetop Kettle or Pot:**
   - Traditional kettles or pots can be used to boil water on a stovetop.
   - Fill the kettle or pot with water and place it on the stove over medium-high heat.
   - Keep an eye on the water and remove it from the heat once it starts boiling.

3. **Microwave:**
   - Microwaves can be used to boil water quickly.
   - Place a microwave-safe container filled with water in the microwave and heat it on high power for 2-3 minutes, depending on the amount of water.
   - Be careful when handling the container as it will be hot.

4. **Campfire or Outdoor Stove:**
   - If you're 

In [ ]:
prompt = "Why is the sky blue?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

 The sky appears blue due to a phenomenon called Rayleigh scattering. This occurs when sunlight, which is composed of all colors of the visible spectrum, interacts with molecules in the Earth's atmosphere, primarily nitrogen and oxygen.

When sunlight enters the atmosphere, it encounters these molecules and particles. The shorter wavelengths of light, such as blue and violet, are more likely to be scattered by these particles than the longer wavelengths, such as red and orange. This is because the shorter wavelengths have a higher frequency and interact more strongly with the molecules and particles in the atmosphere.

As a result, the blue and violet light is scattered in all directions, creating the appearance of a blue sky. The other colors of the spectrum, such as red and orange, are less scattered and continue on their path towards the observer's eyes, contributing to the overall color of the sky.

The amount of scattering depends on the wavelength of light and the density of the 

### Cuidado com as alucinações

Embora os LLMs tenham sido treinados com uma grande quantidade de dados, eles podem gerar textos contendo afirmações não fundamentadas na verdade ou na realidade; essas respostas do LLM são frequentemente chamadas de "alucinações" devido às suas capacidades limitadas de memorização. Observe que simplesmente solicitar que o LLM forneça uma citação não é uma solução para esse problema, pois há casos de LLMs que fornecem citações falsas ou imprecisas. Lidar com alucinações é um desafio fundamental dos LLMs e de uma área de pesquisa em andamento, por isso é importante estar ciente de que os LLMs podem parecer fornecer afirmações confiantes e que parecem corretas, mas que na verdade são incorretas.

Observe que se você pretende usar LLMs para casos de uso criativos, alucinar pode ser bastante útil.

Tente o prompt como o abaixo repetidamente. Você pode notar que às vezes ele dirá com segurança, mas de forma imprecisa, "O primeiro elefante a visitar a lua foi Luna".

In [ ]:
prompt = "Who was the first elephant to visit the moon?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

 No elephant has ever visited the moon.


É evidente que o chatbot está alucinando, já que nenhum elefante jamais voou para a lua. Mas como podemos evitar este tipo de perguntas inadequadas e, mais especificamente, reduzir as alucinações?

Existe um método possível chamado prompt Determine Appropriate Response (DARE), que usa habilmente o próprio LLM para decidir se deve responder a uma pergunta com base em qual é sua missão.

Vamos ver como funciona criando um chatbot para um site de viagens com um leve toque diferente.

In [ ]:
chat_model = ChatModel.from_pretrained("chat-bison@002")

chat = chat_model.start_chat()
dare_prompt = """Remember that before you answer a question, you must check to see if it complies with your mission.
If not, you can say, Sorry I can't answer that question."""

print(
    chat.send_message(
        f"""
Hello! You are an AI chatbot for a travel web site.
Your mission is to provide helpful queries for travelers.

{dare_prompt}
"""
    )
)

MultiCandidateTextGenerationResponse(text=" Hello! I'm here to help you plan your next trip. Whether you're looking for a relaxing beach vacation or an adventurous city getaway, I can help you find the perfect destination and activities for your budget and interests.", _prediction_response=Prediction(predictions=[{'candidates': [{'content': " Hello! I'm here to help you plan your next trip. Whether you're looking for a relaxing beach vacation or an adventurous city getaway, I can help you find the perfect destination and activities for your budget and interests.", 'author': '1'}], 'groundingMetadata': [{}], 'citationMetadata': [{'citations': []}], 'safetyAttributes': [{'blocked': False, 'safetyRatings': [{'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Dangerous Content'}, {'probabilityScore': 0.1, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Harassment'}, {'probabilityScore': 0.0, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'ca

Suponha que façamos uma pergunta simples sobre um dos pontos turísticos mais famosos da Itália.

In [ ]:
prompt = "What is the best place for sightseeing in Milan, Italy?"
print(chat.send_message(prompt))

MultiCandidateTextGenerationResponse(text=" There are many great places for sightseeing in Milan, Italy. Some of the most popular include:\n\n* The Duomo: This stunning Gothic cathedral is one of the most iconic landmarks in Milan. It took nearly 600 years to complete, and its intricate details are truly breathtaking.\n* The Galleria Vittorio Emanuele II: This beautiful shopping arcade is located in the heart of Milan. It's home to some of the most luxurious shops in the world, as well as several cafes and restaurants.\n* The Sforza Castle: This historic castle was once the home of the ruling Sforza family. Today, it houses several museums, including the Pinacoteca di Brera, which features a collection of Renaissance and Baroque art.\n* The Parco Sempione: This large park is located just outside the city center. It's a great place to relax and enjoy the outdoors. There are several playgrounds, a lake, and even a small zoo.\n* The Navigli: This network of canals is a popular spot for lo

Agora vamos fingir que somos um usuário não tão legal e fazer ao chatbot uma pergunta que não tem relação com viagens.


In [ ]:
prompt = "Who was the first elephant to visit the moon?"
print(chat.send_message(prompt))

MultiCandidateTextGenerationResponse(text=" Sorry, I can't answer that question. There have been no elephants on the moon.", _prediction_response=Prediction(predictions=[{'candidates': [{'content': " Sorry, I can't answer that question. There have been no elephants on the moon.", 'author': 'bot'}], 'groundingMetadata': [{}], 'citationMetadata': [{'citations': []}], 'safetyAttributes': [{'blocked': False, 'safetyRatings': [{'probabilityScore': 0.0, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Dangerous Content'}, {'probabilityScore': 0.1, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Harassment'}, {'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Hate Speech'}, {'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Sexually Explicit'}], 'scores': [0.1, 0.1, 0.3, 0.2, 0.1, 0.1, 0.1], 'categories': ['Death, Harm & Tragedy', 'Derogatory', 'Finance', 'Health', 'Insult', 'Legal', 'Sexual']}]}]

Você pode ver que o prompt do DARE adicionou uma camada de 'safetyAttributes' que evitou que o chatbot se desviasse do curso.

### Transforme tarefas generativas em tarefas de classificação para reduzir a variabilidade de resultados

#### Tarefas generativas levam a maior variabilidade de produção


A solicitação abaixo resulta em uma resposta aberta, útil para brainstorming, mas a resposta é altamente variável.

In [ ]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### As tarefas de classificação reduzem a variabilidade de saída

O prompt abaixo resulta em uma escolha e pode ser útil se você quiser que a saída seja mais fácil de controlar.

In [ ]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### Melhore a qualidade da resposta incluindo exemplos


Outra maneira de melhorar a qualidade da resposta é adicionar exemplos ao seu prompt. O LLM aprende no contexto a partir dos exemplos sobre como responder. Normalmente, um a cinco exemplos (disparos) são suficientes para melhorar a qualidade das respostas. Incluir muitos exemplos pode fazer com que o modelo se ajuste demais aos dados e reduza a qualidade das respostas.

Semelhante ao treinamento de modelo clássico, a qualidade e distribuição dos exemplos são muito importantes. Escolha exemplos que sejam representativos dos cenários que você precisa que o modelo aprenda e mantenha a distribuição dos exemplos (por exemplo, número de exemplos por classe no caso de classificação) alinhada com sua distribuição real.

#### Alerta de disparo zero

Abaixo está um exemplo de prompt zero-shot, onde você não fornece nenhum exemplo ao LLM dentro do próprio prompt.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Alerta de disparo único

Abaixo está um exemplo de solicitação única, onde você fornece um exemplo ao LLM dentro da solicitação para fornecer alguma orientação sobre o tipo de resposta que deseja.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)


#### Solicitação de poucos disparos

Abaixo está um exemplo de prompt de poucas tentativas, onde você fornece alguns exemplos ao LLM dentro do prompt para fornecer alguma orientação sobre o tipo de resposta que deseja.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Escolhendo entre métodos de solicitação de disparo zero, disparo único e poucos disparos


A técnica imediata a ser usada dependerá exclusivamente do seu objetivo. Os prompts de disparo zero são mais abertos e podem fornecer respostas criativas, enquanto os prompts de disparo único e de poucos disparos ensinam o modelo como se comportar para que você possa obter respostas mais previsíveis que sejam consistentes com os exemplos fornecidos.